In [5]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 36.7 MB/s eta 0:00:01tcmalloc: large alloc 1982242816 bytes == 0x299c000 @  0x7fb3cdfc6680 0x7fb3cdfe7824 0x5b3128 0x5bbc90 0x5f714c 0x64d800 0x527022 0x504866 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x5f5ee6 0x56bbe1 0x569d8a 0x5f60c3 0x56cc92 0x569d8a 0x5f60c3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 23.9 MB/s eta 0:00:01tcmalloc: large alloc 2477809664 bytes == 0x78c06000 @  0x7fb3cdfc6680 0x7fb3cdfe6da2 0x5f714c 0x64d800 0x527022 0x504866 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x569d8a 0x5f60c3 0x56bbe1 0x5f5ee6 0x56bbe1 0x569d8a 0x5f60c3 0x56cc92 0x569d8a 0x5f60c3 0

In [4]:
!pip install torchtext==0.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.0 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1


In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy import data
from torchtext.legacy import datasets

import spacy
import numpy as np

import time
import random


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [8]:
sentences = []
tags = []
sentenceText = []
sentenceTag = []

with open ('/content/drive/MyDrive/MCI/UPC-2016.txt','rt') as lines:
    for line in lines:
            splitLine = [i for i in (line.split('\t'))]
            if(len(splitLine)==1):
                    sentences.append(sentenceText)
                    tags.append(sentenceTag)
                    sentenceText = []
                    sentenceTag = []
            else:
                    sentenceText.append(splitLine[0])
                    sentenceTag.append((splitLine[1].split('\n')[0]))

In [9]:
import pandas as pd
list_of_tuples = list(zip(sentences, tags))
df = pd.DataFrame(list_of_tuples,columns=['text', 'tag'])
df.head()

,text,tag
0,"[#, اولین, سیاره, سیاره, خارج, از, منظومه, شمس...","[DELM, ADJ_SUP, N_SING, N_SING, P, P, N_SING, ..."
1,"[طی, سالهای, اخیر, ،, بیش, از, دو, دوجین, سیار...","[P, N_PL, ADJ, DELM, ADJ, P, NUM, N_SING, N_SI..."
2,"[لیکن, ستاره‌شناسان, هرگز, این, سیاره‌ها, را, ...","[CON, N_PL, ADV_NEG, DET, N_PL, CLITIC, ADV, V..."
3,"[چنین, تصور, می‌شد, که, این, لرزش, به, علت, جا...","[ADV, N_SING, V_PA, CON, DET, N_SING, P, N_SIN..."
4,"[در, آخرین, تحقیق, به, عمل, آمده, نیز, ستاره‌ش...","[P, ADJ_SUP, N_SING, P, N_SING, ADJ_INO, CON, ..."


In [10]:
TEXT = data.Field()
TAGS = data.Field(unk_token = None)
fields = [("text", TEXT), ("tag", TAGS)]

In [16]:
import random

dfIndexs = list(range(len(df)))
random.shuffle(dfIndexs)

testRate = int(len(df) * 0.1)

valid_examples = [data.Example.fromlist(i, fields) for i in df.iloc[dfIndexs[:testRate]].values.tolist()]
valid_data = data.Dataset(valid_examples, fields)

test_examples = [data.Example.fromlist(i, fields) for i in df.iloc[dfIndexs[testRate:2*testRate]].values.tolist()]
test_data = data.Dataset(test_examples, fields)

train_examples = [data.Example.fromlist(i, fields) for i in df.iloc[dfIndexs[2*testRate:]].values.tolist()]
train_data = data.Dataset(train_examples, fields)

In [17]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 34904
Number of validation examples: 4362
Number of testing examples: 4362


In [22]:
print(vars(train_data.examples[1]))

{'text': ['بحق', 'می\u200cتوان', 'ادعا', 'کرد', 'که', 'دفاع', 'یک\u200cجانبه', 'از', 'آزادی', 'مخالفان', 'جمهوری', 'اسلامی', 'و', 'تسهیل', 'غوغاسالاری', 'اصلاح\u200cطلبان', 'آمریکایی', 'و', 'در\u200cحقیقت', 'جماعتی', 'که', 'در', 'مقام', 'براندازی', 'فرهنگی', 'جمهوری', 'اسلامی', 'ایران', 'هستند', '،', 'یک', 'گزینهٔ', 'معوج', 'و', 'نمونهٔ', 'اصلاحات', 'نامعقول', 'است', '.'], 'tag': ['ADV', 'V_AUX', 'N_SING', 'V_PA', 'CON', 'N_SING', 'ADJ', 'P', 'N_SING', 'N_PL', 'N_SING', 'ADJ', 'CON', 'N_SING', 'N_SING', 'N_PL', 'ADJ', 'CON', 'CON', 'N_PL', 'CON', 'P', 'N_SING', 'N_SING', 'ADJ', 'N_SING', 'ADJ', 'N_SING', 'V_PRS', 'DELM', 'NUM', 'N_SING', 'ADJ', 'CON', 'N_SING', 'N_PL', 'ADJ', 'V_PRS', 'DELM']}


In [23]:
from torchtext.vocab import Vectors

vectors = Vectors(name='/content/drive/MyDrive/MCI/cc.fa.300.vec')

MIN_FREQ = 2

TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ,
                 vectors = vectors,
                 unk_init = torch.Tensor.normal_
                 )


TAGS.build_vocab(train_data)


100%|██████████| 2000000/2000000 [02:58<00:00, 11208.15it/s]


In [24]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in TAGS vocabulary: {len(TAGS.vocab)}")

Unique tokens in TEXT vocabulary: 24389
Unique tokens in TAGS vocabulary: 32


In [62]:
print(TAGS.vocab.stoi)

defaultdict(None, {'<pad>': 0, 'N_SING': 1, 'P': 2, 'DELM': 3, 'ADJ': 4, 'CON': 5, 'N_PL': 6, 'V_PRS': 7, 'V_PA': 8, 'NUM': 9, 'PRO': 10, 'DET': 11, 'CLITIC': 12, 'V_PP': 13, 'V_SUB': 14, 'ADV': 15, 'ADV_TIME': 16, 'V_AUX': 17, 'ADJ_SUP': 18, 'ADJ_CMPR': 19, 'FW': 20, 'ADV_I': 21, 'ADV_NEG': 22, 'ADJ_INO': 23, 'ADV_COMP': 24, 'ADV_LOC': 25, 'V_IMP': 26, 'PREV': 27, 'INT': 28, 'N_VOC': 29, '': 30, 'ADJ_VOC': 31})


In [25]:
print(TEXT.vocab.freqs.most_common(20))

[('و', 47555), ('،', 36043), ('در', 33914), ('.', 32553), ('به', 27566), ('از', 23926), ('که', 20647), ('این', 17517), ('را', 14989), ('است', 14295), ('با', 11236), (':', 9926), ('-', 7407), ('آن', 5533), ('برای', 5103), ('یک', 4835), ('خود', 4564), ('بر', 4446), ('کرد', 3910), ('ایران', 3700)]


In [26]:
def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
        
    return tag_counts_percentages

In [27]:
print("Tag\t\tCount\t\tPercentage\n")

for tag, count, percent in tag_percentage(TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t\t{count}\t\t{percent*100:4.1f}%")

Tag		Count		Percentage

N_SING		348297		34.0%
P		118772		11.6%
DELM		97763		 9.5%
ADJ		92006		 9.0%
CON		82784		 8.1%
N_PL		61091		 6.0%
V_PRS		36739		 3.6%
V_PA		31453		 3.1%
NUM		28294		 2.8%
PRO		27393		 2.7%
DET		21462		 2.1%
CLITIC		14991		 1.5%
V_PP		14072		 1.4%
V_SUB		12920		 1.3%
ADV		10777		 1.1%
ADV_TIME		6003		 0.6%
V_AUX		4955		 0.5%
ADJ_SUP		2987		 0.3%
ADJ_CMPR		2450		 0.2%
FW		1742		 0.2%
ADV_I		1730		 0.2%
ADV_NEG		1714		 0.2%
ADJ_INO		1666		 0.2%
ADV_COMP		1077		 0.1%
ADV_LOC		904		 0.1%
V_IMP		643		 0.1%
PREV		342		 0.0%
INT		291		 0.0%
N_VOC		27		 0.0%
		9		 0.0%
ADJ_VOC		1		 0.0%


In [28]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device,
    sort_key=lambda x: len(x.text)
    )

In [29]:
class BiLSTMPOSTagger(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_dim, 
                            num_layers = n_layers, 
                            bidirectional = bidirectional,
                            dropout = dropout if n_layers > 1 else 0)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):

        embedded = self.dropout(self.embedding(text))
        outputs, (hidden, cell) = self.lstm(embedded)
        predictions = self.fc(self.dropout(outputs))
        
        return predictions


In [30]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 128
OUTPUT_DIM = len(TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = BiLSTMPOSTagger(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT, 
                        PAD_IDX)

In [31]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)
        
model.apply(init_weights)

BiLSTMPOSTagger(
  (embedding): Embedding(24389, 300, padding_idx=1)
  (lstm): LSTM(300, 128, num_layers=2, dropout=0.25, bidirectional=True)
  (fc): Linear(in_features=256, out_features=32, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [32]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,160,508 trainable parameters


In [33]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([24389, 300])


In [34]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1988, -0.0852,  0.0563,  ..., -0.0996,  0.0842,  0.0612],
        ...,
        [-0.1217, -0.0713, -0.0802,  ..., -0.0771, -0.0419, -0.1008],
        [-0.0021, -0.0273, -0.0190,  ..., -0.0124,  0.0245, -0.0037],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [35]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1988, -0.0852,  0.0563,  ..., -0.0996,  0.0842,  0.0612],
        ...,
        [-0.1217, -0.0713, -0.0802,  ..., -0.0771, -0.0419, -0.1008],
        [-0.0021, -0.0273, -0.0190,  ..., -0.0124,  0.0245, -0.0037],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])


In [36]:
optimizer = optim.Adam(model.parameters())

In [37]:
TAG_PAD_IDX = TAGS.vocab.stoi[TAGS.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)   

In [38]:
model = model.to(device)
criterion = criterion.to(device)

In [39]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / y[non_pad_elements].shape[0]

In [41]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
   
        text = batch.text
        tags = batch.tag
        
        optimizer.zero_grad()
        
        
        predictions = model(text)
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        
        loss = criterion(predictions, tags)
                
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [42]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.text
            tags = batch.tag
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [43]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [44]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/MCI/posTag-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 17s
	Train Loss: 0.584 | Train Acc: 84.15%
	 Val. Loss: 0.141 |  Val. Acc: 95.75%
Epoch: 02 | Epoch Time: 0m 17s
	Train Loss: 0.104 | Train Acc: 96.85%
	 Val. Loss: 0.115 |  Val. Acc: 96.33%
Epoch: 03 | Epoch Time: 0m 17s
	Train Loss: 0.080 | Train Acc: 97.44%
	 Val. Loss: 0.108 |  Val. Acc: 96.58%
Epoch: 04 | Epoch Time: 0m 20s
	Train Loss: 0.067 | Train Acc: 97.82%
	 Val. Loss: 0.109 |  Val. Acc: 96.54%
Epoch: 05 | Epoch Time: 0m 18s
	Train Loss: 0.056 | Train Acc: 98.17%
	 Val. Loss: 0.111 |  Val. Acc: 96.56%
Epoch: 06 | Epoch Time: 0m 18s
	Train Loss: 0.046 | Train Acc: 98.47%
	 Val. Loss: 0.118 |  Val. Acc: 96.50%
Epoch: 07 | Epoch Time: 0m 18s
	Train Loss: 0.038 | Train Acc: 98.74%
	 Val. Loss: 0.129 |  Val. Acc: 96.49%
Epoch: 08 | Epoch Time: 0m 18s
	Train Loss: 0.031 | Train Acc: 98.97%
	 Val. Loss: 0.133 |  Val. Acc: 96.49%
Epoch: 09 | Epoch Time: 0m 18s
	Train Loss: 0.026 | Train Acc: 99.16%
	 Val. Loss: 0.145 |  Val. Acc: 96.45%
Epoch: 10 | Epoch T

In [45]:
model.load_state_dict(torch.load('/content/drive/MyDrive/MCI/posTag-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.108 |  Test Acc: 96.47%


In [72]:
def tag_sentence(model, device, sentence, text_field, tag_field):
    
    model.eval()
    
    if isinstance(sentence, str):
        tokens = sentence.strip().split(' ')
    else:
        tokens = [token for token in sentence]

    if text_field.lower:
        tokens = [t.lower() for t in tokens]
        
    numericalized_tokens = [text_field.vocab.stoi[t] for t in tokens]

    unk_idx = text_field.vocab.stoi[text_field.unk_token]
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
         
    predictions = model(token_tensor)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    return tokens, predicted_tags, unks

In [73]:
example_index = 1

sentence = vars(train_data.examples[example_index])['text']
actual_tags = vars(train_data.examples[example_index])['tag']

print(sentence)

['بحق', 'می\u200cتوان', 'ادعا', 'کرد', 'که', 'دفاع', 'یک\u200cجانبه', 'از', 'آزادی', 'مخالفان', 'جمهوری', 'اسلامی', 'و', 'تسهیل', 'غوغاسالاری', 'اصلاح\u200cطلبان', 'آمریکایی', 'و', 'در\u200cحقیقت', 'جماعتی', 'که', 'در', 'مقام', 'براندازی', 'فرهنگی', 'جمهوری', 'اسلامی', 'ایران', 'هستند', '،', 'یک', 'گزینهٔ', 'معوج', 'و', 'نمونهٔ', 'اصلاحات', 'نامعقول', 'است', '.']


In [74]:
tokens, pred_tags, unks = tag_sentence(model, 
                                       device, 
                                       sentence, 
                                       TEXT, 
                                       TAGS)

print(unks)

[]


In [75]:
print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

Pred. Tag	Actual Tag	Correct?	Token

ADV		ADV		✔		بحق
V_AUX		V_AUX		✔		می‌توان
N_SING		N_SING		✔		ادعا
V_PA		V_PA		✔		کرد
CON		CON		✔		که
N_SING		N_SING		✔		دفاع
ADJ		ADJ		✔		یک‌جانبه
P		P		✔		از
N_SING		N_SING		✔		آزادی
N_PL		N_PL		✔		مخالفان
N_SING		N_SING		✔		جمهوری
ADJ		ADJ		✔		اسلامی
CON		CON		✔		و
N_SING		N_SING		✔		تسهیل
N_SING		N_SING		✔		غوغاسالاری
N_PL		N_PL		✔		اصلاح‌طلبان
ADJ		ADJ		✔		آمریکایی
CON		CON		✔		و
CON		CON		✔		در‌حقیقت
N_PL		N_PL		✔		جماعتی
CON		CON		✔		که
P		P		✔		در
N_SING		N_SING		✔		مقام
N_SING		N_SING		✔		براندازی
ADJ		ADJ		✔		فرهنگی
N_SING		N_SING		✔		جمهوری
ADJ		ADJ		✔		اسلامی
N_SING		N_SING		✔		ایران
V_PRS		V_PRS		✔		هستند
DELM		DELM		✔		،
NUM		NUM		✔		یک
N_SING		N_SING		✔		گزینهٔ
ADJ		ADJ		✔		معوج
CON		CON		✔		و
ADJ		N_SING		✘		نمونهٔ
N_PL		N_PL		✔		اصلاحات
ADJ		ADJ		✔		نامعقول
V_PRS		V_PRS		✔		است
DELM		DELM		✔		.


In [76]:
sentence = 'سلام و درود بر مردم ایران'

tokens, tags, unks = tag_sentence(model, 
                                  device, 
                                  sentence, 
                                  TEXT, 
                                  TAGS)

print(unks)

[]


In [77]:
print("Pred. Tag\tToken\n")

for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")

Pred. Tag	Token

N_SING		سلام
CON		و
N_SING		درود
P		بر
N_SING		مردم
N_SING		ایران
